In [1]:
! nvidia-smi

Tue Jul 16 14:55:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           On  | 00000000:86:00.0 Off |                    0 |
| N/A   33C    P0              46W / 250W |  31972MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Profiling

In [2]:
%load_ext memory_profiler
%load_ext line_profiler

In [3]:
import cProfile
import pstats
import io
from memory_profiler import memory_usage
from memory_profiler import profile

In [4]:
def profile_performance(func, *args, lines=10, **kwargs):
    """
    Profiler les performances d'une fonction.
    
    Parameters:
    func (callable): La fonction à profiler.
    *args: Les arguments positionnels pour la fonction.
    lines (int): Nombre de lignes à afficher dans les résultats.
    **kwargs: Les arguments nommés pour la fonction.
    
    Returns:
    result: Le résultat de la fonction exécutée.
    """
    pr = cProfile.Profile()
    pr.enable()
    result = func(*args, **kwargs)
    pr.disable()
    
    s = io.StringIO()
    ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
    ps.print_stats(lines)  # Afficher uniquement les 'lines' premières lignes
    
    profile_output = s.getvalue()
    print(profile_output)
    
    with open("profile_performance_result.txt", "w") as f:
        f.write(profile_output)
    
    return result

In [5]:
def profile_memory(func, *args, **kwargs):
    """
    Profiler la mémoire utilisée par une fonction.
    
    Parameters:
    func (callable): La fonction à profiler.
    *args: Les arguments positionnels pour la fonction.
    **kwargs: Les arguments nommés pour la fonction.
    
    Returns:
    result: Le résultat de la fonction exécutée.
    """
    mem_usage = memory_usage((func, args, kwargs))
    
    print(mem_usage)
    
    with open("profile_memory_result.txt", "w") as f:
        for usage in mem_usage:
            f.write(f"{usage}\n")
    
    result = func(*args, **kwargs)
    
    return result

# Importation des données

In [6]:
import ztfimg
from ztfimg import catalog as catalog
import ztfin2p3
from ztfin2p3 import catalog
import pandas
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax.scipy import stats as jstats
from jax import vmap
from scipy.ndimage import zoom

In [7]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [8]:
img1 = ztfimg.ScienceQuadrant.from_filename("/sps/ztf/data/sci/2020/0924/431759/ztf_20200924431759_000655_zr_c13_o_q3_sciimg.fits")

q1 = img1.get_ccd().get_quadrant(1) #selects quadrant 1 
qimg1 = q1.get_data()  #converted to numpy array

# importing data into a pandas.dataframe
qimg1_catalog = ztfin2p3.catalog.get_img_refcatalog(q1, which="gaia_dr2") # selects the data corresponding to the quadrant in the gaia_dr2 catalog
qimg1_catalog['isolated'] = ztfimg.catalog.get_isolated(qimg1_catalog, seplimit=20) #select stars that are 15 arcsec apart and add a Boolean column
qimg_catalog_isolated = qimg1_catalog.loc[qimg1_catalog['isolated'] == True] # we keep only isolated stars (whose Boolean is True)
qimg_catalog_isolated = qimg_catalog_isolated.drop('isolated', axis=1) #supression of isolated column

# magnitude selection
mag_inf = qimg_catalog_isolated.phot_g_mean_mag > 14 #selects magnitudes above 14
mag_sup = qimg_catalog_isolated.phot_g_mean_mag < 18 #selects magnitudes below 18
qimg_catalog_isolated_mag = qimg_catalog_isolated.loc[mag_inf & mag_sup] #application of the mask on magnitudes

# location selection
mag_bord_left = qimg_catalog_isolated_mag.x > 15 # removes stars on the left edge of 15 pixels
mag_bord_right = qimg_catalog_isolated_mag.x < (q1.shape[0]-15) # removes stars on the right edge of 15 pixels
mag_bord_top = qimg_catalog_isolated_mag.y > 15 # removes the stars on the top edge by 15 pixels
mag_bord_bottom = qimg_catalog_isolated_mag.y < (q1.shape[1]-15) # removes stars on the bottom edge of 15 pixels
mag_bord_combined = np.logical_and.reduce((mag_bord_left, mag_bord_right, mag_bord_top, mag_bord_bottom)) #edge selection
qimg_catalog_isolated_mag_bord = qimg_catalog_isolated_mag[mag_bord_combined] #application of edge mask
qimg_catalog_isolated_mag_bord

,id,coord_ra,coord_dec,phot_g_mean_flux,phot_bp_mean_flux,phot_rp_mean_flux,phot_g_mean_fluxErr,phot_bp_mean_fluxErr,phot_rp_mean_fluxErr,coord_raErr,...,ra,dec,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,phot_g_mean_magErr,phot_bp_mean_magErr,phot_rp_mean_magErr,x,y
3292,220210914664501376,1.026161,0.637330,3.411760e+05,2.496884e+05,4.863944e+05,333.605022,4307.633356,1871.193552,0.002145,...,58.794687,36.516333,17.567620,17.906570,17.182594,0.001062,0.018731,0.004177,665.035175,1267.744563
3314,220210949024241920,1.025702,0.637414,2.846405e+05,1.552300e+05,4.988820e+05,263.706599,2006.851979,3996.709121,0.001880,...,58.768417,36.521120,17.764324,18.422627,17.155071,0.001006,0.014037,0.008698,589.417449,1282.097111
3343,220214655577103232,1.022990,0.637796,6.637216e+05,4.199027e+05,1.086072e+06,553.114736,1744.992099,2613.982357,0.001542,...,58.612995,36.543040,16.845101,17.342194,16.310420,0.000905,0.004512,0.002613,142.980478,1344.672348
3494,220216407923781760,1.023645,0.639737,3.310731e+05,2.083656e+05,5.224247e+05,297.967950,1487.596690,3983.500840,0.001617,...,58.650511,36.654212,17.600256,18.103001,17.105006,0.000977,0.007751,0.008279,236.507139,1743.317737
3524,220213216767321088,1.026577,0.638661,7.455134e+05,4.821637e+05,1.142259e+06,524.353589,3058.071864,5102.794374,0.001253,...,58.818547,36.592557,16.718927,17.192079,16.255654,0.000764,0.006886,0.004850,723.407453,1541.004010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45776,220167793192842368,1.033080,0.634488,4.161678e+06,3.407272e+06,5.055197e+06,1242.641362,5762.443728,7624.821103,0.000454,...,59.191138,36.353509,14.851894,15.069049,14.640720,0.000324,0.001836,0.001638,1820.108284,731.855584
45799,220175077457571712,1.036926,0.636100,1.675470e+06,1.183528e+06,2.416099e+06,681.649828,2936.846731,3462.371170,0.000738,...,59.411462,36.445829,15.839724,16.217119,15.442279,0.000442,0.002694,0.001556,2436.769333,1085.493345
45891,220225586272711168,1.030969,0.639192,6.834436e+05,4.806962e+05,9.924018e+05,407.310306,1946.126256,4175.423717,0.001170,...,59.070190,36.622983,16.813309,17.195389,16.408347,0.000647,0.004396,0.004568,1437.085775,1675.756830
45903,220222596975471872,1.031456,0.639021,6.083751e+05,5.244016e+05,7.197826e+05,425.536839,2348.414058,4584.703300,0.001287,...,59.098059,36.613195,16.939637,17.100906,16.757062,0.000759,0.004862,0.006916,1517.911040,1644.046424


# Moffat sans pixel grille

In [9]:
def get_stamps(dataframe, size=17):
    """
    Parameters:
    ----------
    dataframe: pandas.dataframe
        dataframe of stars's data from Gaia dr2
    size: int
        stamp size (17,17)

    Returns:
    --------
    stamps: np.asarray
        data of the star image
    """
    stamps = []
    for index, df in qimg_catalog_isolated_mag_bord.iterrows():
        x0 = int(round(df["x"]))
        y0 = int(round(df["y"]))
        left = x0 - (size // 2)
        top = y0 - (size // 2)
        right = left + size
        bottom = top + size
        stamps.append(qimg1[top:bottom, left:right])
    return np.asarray(stamps)


def moffat(x, y, x0, y0, A, alpha, gamma):
    r_squared = (x - x0)**2 + (y - y0)**2
    return A * (1 + (r_squared / gamma**2))**(-alpha)


@jax.jit
def get_model(params):
    """
    Returns the Moffat function

    Parameters:
    -----------
    params: pytree
        [x0,y0: (2,N) centroids
         a: (N,) amplitudes
         b: (N,) backgrounds
         gamma: (1,) float
         alpha: (1,) float]
    
    Returns:
    -------
    model : arraylike
        the Moffat function
    """
    mu, A, b, alpha, gamma = params
    x0, y0 = mu[:, 0], mu[:, 1]
    
    vectorized_moffat = vmap(moffat, in_axes=(None, None, 0, 0, 0, None, None)) # to vectorize on centroids
    norm = vectorized_moffat(pos[:, 0], pos[:, 1], x0, y0, A, alpha, gamma)
    norm_model = norm + b[:, None]
    
    return norm_model
    
@jax.jit
def get_likelihood(params):
    """
    Computes the Chi squared from the selected model 

    Parameters:
    ----------
    params: list
        selected model parameters
    data: arraylike
        images of stars (flattened)
    pos: arraylike
        The positions (meshgrid) where the model is evaluated.

    Returns:
    --------
    summ: float
        chi squared sum for all stars
    """
    model = get_model(params)
    summ = jnp.sum((model - data)**2)
    return summ


@jax.jit
def get_logprior(params):
    """
    Returns the probability to have gamma and alpha
    
    Parameters:
    -----------
    params: list
        selected model parameters

    X: arraylike
        The X coordinates.

    Y: arraylike
        The Y coordinates.
    
    Returns:
    -------
    logprior: float
        sum of the two sigma probabilities
    """
    mu, A, b, alpha, gamma = params
    logprior_alpha = jstats.norm.logpdf(alpha, loc=1.0, scale=0.5) 
    logprior_gamma = jstats.norm.logpdf(gamma, loc=1.0, scale=0.5) 
    logprior = logprior_alpha + logprior_gamma
    return logprior

@jax.jit
def get_logprob(params):
    """ 
    Computes the sum of the gamma and alpha probabilities and the chi squared

    Parameters:
    -----------
    params: list
        selected model parameters
    data: arraylike
        images of stars (flattened)
    pos: arraylike
        The positions (meshgrid) where the model is evaluated.

    Returns:
    --------
    logprob: float
        sum of the sigma probabilities and the chi squared
    """
    logprior = -1 * get_logprior(params)  # to minimize
    likelihood = get_likelihood(params)
    logprob = logprior + likelihood
    return logprob

In [10]:
def fit_tncg(func, init_param, 
             niter=10, tol=5e-3, 
             lmbda=1e2, 
             **kwargs):
    """ Hessian-free second order optimization algorithm

    The following implementation of TN-CG is largely based on
    recommendations given in Martens, James (2010, Deep learning via
    Hessian-free optimization, Proc. International  Conference on
    Machine Learning).

    Parameters
    ----------
    func: function
        function to minimize. Should return a float.

    init_param: 
        entry parameter of the input func

    niter: int
        maximum number of iterations

    tol: float
        targeted func variations below which the iteration will stop

    lmbda: float
        lambda parameter of the tncg algorithm. (optstate)

    **kwargs other func entries 

    Returns
    -------
    list
        - best parameters
        - loss (array)

    Example
    -------
    ```python
    import jax
    from edris import simulation, minimize
    key = jax.random.PRNGKey(1234)
    truth, simu = simulation.get_simple_simulation(key, size=1_000)

    def get_total_chi2(param, data):
        # model for a line with error on both axes but no intrinsic scatter.
        x_model = param["x_model"]
        y_model = x_model * param["a"] + param["b"]
    
        chi2_y = jnp.sum( ((data["x_obs"] - x_model)/data["x_err"])**2 )
        chi2_x = jnp.sum( ((data["y_obs"] - y_model)/data["y_err"])**2 )
    
        return chi2_y + chi2_x

    init_param = {"a": 8., "b":0., "x_model": simu["x_obs"]} # careful, must be float
    best_params, loss = minimize.fit_tncg(get_total_chi2, init_param, data=simu)
    ```
    
    """
    # handle kwargs more easily
    func_ = lambda x: func(x, **kwargs)
    fg = jax.value_and_grad(func_)
    
    # - internal function --- #
    def hessian_vector_product(g, x, v):
        return jax.jvp(g, (x,), (v,))[1]

    def step_tncg(x, optstate):
        loss, grads = fg(x)
        lmbda = optstate['lmbda']
        fvp = lambda v: jax.tree_util.tree_map(lambda d1, d2: d1 + lmbda*d2, hessian_vector_product(jax.grad(func_), x, v), v)
        updates, _ = jax.scipy.sparse.linalg.cg(fvp, grads, maxiter=50)
        coco = jax.tree_util.tree_reduce(lambda x, y: x+y, jax.tree_util.tree_map(lambda x, y: (-x*y).sum(), grads, updates))
        return updates, loss, optstate, coco

    step_tncg = jax.jit( step_tncg )
    # ----------------------- #
    
    x = init_param
    optstate = {'lmbda': lmbda}
    losses = []

    for i in range(niter):
        updates, loss, optstate, coco = step_tncg(x, optstate)
        x1 = jax.tree_util.tree_map(lambda x, y: x - y, x, updates)
        dloss = func_(x1) - loss
        losses.append(loss)
        rho = dloss / coco
        
        if rho < 0.25:
            optstate['lmbda'] = optstate['lmbda'] * 1.5
        elif rho > 0.75:
            optstate['lmbda'] = optstate['lmbda'] * 0.3
            
        if dloss < 0: # accept the step
            x = x1
            
        if tol is not None and dloss > -tol:
            break
        
    return x, losses

In [11]:
stamps = get_stamps(qimg_catalog_isolated_mag_bord)
coefs = np.sum(stamps, axis=(1,2))
stamps/=coefs[:,None, None]

nstars = len(stamps)
size= 17
X = jnp.linspace(-size/2, size/2, size)
Y = jnp.linspace(-size/2, size/2, size)
X, Y = jnp.meshgrid(X, Y)
pos = jnp.vstack((X.ravel(), Y.ravel())).T

# guess
x0 = jnp.zeros((nstars,), dtype="float32")
y0 = jnp.zeros((nstars,), dtype="float32")
mu = jnp.vstack([x0,y0]).T
A = jnp.ones((nstars,), dtype="float32")  
b = jnp.zeros((nstars,), dtype="float32")
alpha = jnp.array(1., dtype="float32")  
gamma = jnp.array(1., dtype="float32") 

data = stamps.reshape(len(stamps), -1)
grad_func = jax.jit(jax.grad(get_logprob)) # get the derivative

2024-07-16 14:55:09.750860: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [12]:
guess = [mu, A, b, alpha, gamma]
tncg_params, tncg_loss = fit_tncg(get_logprob, guess, tol=1e-5, niter=50, lmbda=10000)

print("Profilage de la performance")
result_perf = profile_performance(fit_tncg, get_logprob, guess, tol=1e-5, niter=50, lmbda=10000, lines=10)
print("Profilage de la mémoire en Mo")
result_mem = profile_memory(fit_tncg, get_logprob, guess, tol=1e-5, niter=50, lmbda=10000)

Profilage de la performance
         928064 function calls (911449 primitive calls) in 3.290 seconds

   Ordered by: cumulative time
   List reduced from 1201 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.075    0.075    3.309    3.309 /tmp/ipykernel_43109/2656809650.py:1(fit_tncg)
    684/2    0.001    0.000    3.172    1.586 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/traceback_util.py:175(reraise_with_filtered_traceback)
    656/2    0.003    0.000    3.172    1.586 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/pjit.py:296(cache_miss)
    656/2    0.007    0.000    3.171    1.586 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/pjit.py:165(_python_pjit_helper)
  240/124    0.002    0.000    3.158    0.025 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/profiler.py:332(wrapper)
  724/674    0.003  

/pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[835.1484375, 835.16015625, 835.16015625, 835.16015625, 835.28515625, 835.28515625, 835.69921875, 835.69921875, 835.69921875, 835.69921875, 835.69921875, 835.69921875, 835.69921875, 835.69921875, 835.59375, 835.6015625, 835.6015625, 836.62890625, 836.62890625, 836.671875, 836.671875, 836.671875, 836.89453125, 836.89453125, 836.89453125, 836.89453125, 836.89453125, 836.7734375, 836.83203125]


# Pixel grille pour toutes les étoiles du dataframe

In [13]:
def get_stamps(dataframe, size=17):
    """
    Parameters:
    ----------
    dataframe: pandas.dataframe
        dataframe of stars's data from Gaia dr2
    size: int
        stamp size (17,17)

    Returns:
    --------
    stamps: np.asarray
        data of the star image
    """
    stamps = []
    for index, df in qimg_catalog_isolated_mag_bord.iterrows():
        x0 = int(round(df["x"]))
        y0 = int(round(df["y"]))
        left = x0 - (size // 2)
        top = y0 - (size // 2)
        right = left + size
        bottom = top + size
        stamps.append(qimg1[top:bottom, left:right])
    return np.asarray(stamps)

In [14]:
stamps = get_stamps(qimg_catalog_isolated_mag_bord)
coefs = np.sum(stamps, axis=(1,2))
stamps/=coefs[:,None, None]
coords = jnp.array(qimg_catalog_isolated_mag_bord[['x', 'y']])

print("Profilage de la performance")
result_perf = profile_performance(get_stamps, qimg_catalog_isolated_mag_bord, lines=10)
print("Profilage de la mémoire en Mo")
result_mem = profile_memory(get_stamps, qimg_catalog_isolated_mag_bord)

Profilage de la performance
         70106 function calls (68210 primitive calls) in 0.061 seconds

   Ordered by: cumulative time
   List reduced from 105 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.003    0.003    0.061    0.061 /tmp/ipykernel_43109/3365904432.py:1(get_stamps)
      474    0.001    0.000    0.042    0.000 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/pandas/core/frame.py:1401(iterrows)
      473    0.005    0.000    0.037    0.000 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/pandas/core/series.py:371(__init__)
      946    0.003    0.000    0.012    0.000 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/pandas/core/series.py:1016(__getitem__)
      473    0.003    0.000    0.008    0.000 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/pandas/core/construction.py:519(sanitize_array)
      946    0.002    0.000    0.0

/pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[838.1015625, 838.1015625, 838.1015625, 838.1015625, 838.1015625, 838.1015625, 838.1015625, 838.1015625]


In [15]:
def calculate_grid_overlap(coords, stamps, size, grid_size, pixel_size_to_fit, pixel_size_grid):
    half_size = size // 2
    left = coords[:, 0] - half_size
    top = coords[:, 1] - half_size
    right = left + size
    bottom = top + size
    
    x_to_fit = jnp.linspace(-half_size, half_size, size)
    y_to_fit = jnp.linspace(-half_size, half_size, size)
    X_to_fit, Y_to_fit = jnp.meshgrid(x_to_fit, y_to_fit)
    
    x_grid = jnp.linspace(-half_size, half_size, grid_size)
    y_grid = jnp.linspace(-half_size, half_size, grid_size)
    X_grid, Y_grid = jnp.meshgrid(x_grid, y_grid)
    pos = jnp.vstack((X_grid.ravel(), Y_grid.ravel())).T
    
    X_to_fit_start = X_to_fit[:, :, None, None]
    Y_to_fit_start = Y_to_fit[:, :, None, None]
    X_to_fit_end = X_to_fit_start + pixel_size_to_fit
    Y_to_fit_end = Y_to_fit_start + pixel_size_to_fit
    
    X_grid_start = X_grid[None, None, :, :]
    Y_grid_start = Y_grid[None, None, :, :]
    X_grid_end = X_grid_start + pixel_size_grid
    Y_grid_end = Y_grid_start + pixel_size_grid
    
    X_overlap = jnp.maximum(0, jnp.minimum(X_to_fit_end, X_grid_end) - jnp.maximum(X_to_fit_start, X_grid_start))
    Y_overlap = jnp.maximum(0, jnp.minimum(Y_to_fit_end, Y_grid_end) - jnp.maximum(Y_to_fit_start, Y_grid_start))
    
    overlap_area = X_overlap * Y_overlap
    proportion_overlap = overlap_area / pixel_size_to_fit
    
    grids = jnp.array([
        jnp.sum(proportion_overlap * stamps[i, :, :, None, None], axis=(0, 1))
        for i in range(stamps.shape[0])
    ])
    
    percentages = grids / grids.sum(axis=(1, 2), keepdims=True)
    return percentages, pos

In [16]:
size = 17 #taille de l'image to_fit
pixel_size_to_fit = 1 #taille du pixel de to_fit
grid_size = 19 #taille de l'image grid
pixel_size_grid = 1.8 #taille du pixel de grid

percentages, pos = calculate_grid_overlap(coords, stamps, size, grid_size, pixel_size_to_fit, pixel_size_grid)

print("Profilage de la performance")
result_perf = profile_performance(calculate_grid_overlap, coords, stamps, size, grid_size, pixel_size_to_fit, pixel_size_grid, lines=10)
print("Profilage de la mémoire en Mo")
result_mem = profile_memory(calculate_grid_overlap, coords, stamps, size, grid_size, pixel_size_to_fit, pixel_size_grid)

Profilage de la performance
         196589 function calls (196116 primitive calls) in 0.260 seconds

   Ordered by: cumulative time
   List reduced from 171 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.260    0.260 /tmp/ipykernel_43109/3818989536.py:1(calculate_grid_overlap)
    478/5    0.006    0.000    0.142    0.028 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/numpy/lax_numpy.py:2101(array)
        1    0.005    0.005    0.098    0.098 /tmp/ipykernel_43109/3818989536.py:33(<listcomp>)
        1    0.002    0.002    0.087    0.087 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/numpy/lax_numpy.py:1823(stack)
      486    0.059    0.000    0.060    0.000 /pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/site-packages/jax/_src/numpy/array_methods.py:259(deferring_binary_op)
      477    0.001    0.000    0.057    0.000 /pbs/throng/zt

/pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/pbs/throng/ztf/anaconda3/envs/zenv11/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[839.21484375, 839.21484375, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875, 839.21875]


## fit Gaussienne symetric

In [17]:
@jax.jit
def get_model_gauss(params):
    """
    returns the normalized Gaussian function
    =>
    Parameters:
    -----------
    params: pytree
        [mu: (2,N) centroids
         a: (N,) amplitudes
         b: (N,) backgrounbds
         sigmax: (1,) float
         sigmay: (1,) float
         ]
    Returns:
    -------
    model : arraylike
        the normalized Gaussian function
    """
    mu, a, b, sigmax, sigmay = params
    
    cov = jnp.asarray(([sigmax**2 , 0] , [0, sigmay**2]), dtype="float32")
    norm = jstats.multivariate_normal.pdf(pos[None,:],  # same structure for all stars
                                      mean = mu[:,None], # one position per star
                                      cov = cov[None,:] # same cov for all stars
                                     )
    norm_model =  a[:,None]*norm + b[:,None] # (N, size*size)
    return norm_model

@jax.jit
def get_likelihood_gauss(params):
    """
    Computes the Chi squared from selected model 
    
    Parameters:
    ----------
    params: list
        selected model parameters
    stamp: arraylike
        image of one star
    
    Returns:
    --------
    summ: float
        chi squared
    """
    model = get_model_gauss(params)
    summ = jnp.sum((model - data)**2)
    return summ


@jax.jit
def get_logprior_gauss(params):
    """
    Returns the probability to have sigmax and sigma y
    
    Parameters:
    -----------
    params: list
        selected model parameters
    
    Returns:
    -------
    logprior: float
        sum of the two sigma probabilities
    """
    mu, a, b, sigmax, sigmay = params
    logprior_sx = jstats.norm.logpdf(sigmax, loc=1, scale=0.5) # float
    logprior_sy = jstats.norm.logpdf(sigmay, loc=1, scale=0.5) # float
    logprior = logprior_sx + logprior_sy # get the maximize
    return logprior

@jax.jit
def get_logprob_gauss(params):
    """ 
    Computes the sum of the sigma probabilities and the chi squared
    
    Parameters:
    -----------
    params: list
        selected model parameters
    stamp: arraylike
        image of one star
    
    Returns:
    logprob: float
        sum of the sigma probabilities and the chi squared
    """
    logprior = -1* get_logprior_gauss(params) # to minimize
    likelihood = get_likelihood_gauss(params)
    logprob = logprior + likelihood
    return logprob

In [18]:
nstars = len(percentages)

# guess
x0 = jnp.zeros((nstars,), dtype="float32")
y0 = jnp.zeros((nstars,), dtype="float32")
mu = jnp.vstack([x0,y0]).T
sigmax, sigmay = 1., 1.
cov = jnp.asarray(([sigmax**2 , 0] , [0, sigmay**2]), dtype="float32")
a = jnp.ones((nstars,), dtype="float32")
b = jnp.zeros((nstars,), dtype="float32")

data = percentages.reshape(len(percentages), -1)
grad_func = jax.jit(jax.grad(get_logprob_gauss)) # get the derivative

In [19]:
guess_gauss = [mu, a, b, sigmax, sigmay]
tncg_params_gauss, tncg_loss_gauss = fit_tncg(get_logprob_gauss, guess_gauss, tol=1e-5, niter=50, lmbda=10000)

print("Profilage de la performance")
result_perf = profile_performance(fit_tncg, get_logprob_gauss, guess_gauss, tol=1e-5, niter=50, lmbda=10000, lines=10)
print("Profilage de la mémoire en Mo")
result_mem = profile_memory(fit_tncg, get_logprob_gauss, guess_gauss, tol=1e-5, niter=50, lmbda=10000)

XlaRuntimeError: INTERNAL: cuSolver internal error

## fit Gaussienne elliptic

In [ ]:
def gauss_elliptic(x, y, x0, y0, A, sigmax, sigmay):
    x_sqrd = (x - x0)**2
    y_sqrd = (y - y0)**2
    exponent = -(x_sqrd / (2*sigmax**2)) - (y_sqrd / (2*sigmay**2))
    return A * jnp.exp(exponent)

def get_model_gauss_elliptic(params):

    mu, A, b, sigmax, sigmay = params
    x0, y0 = mu[:, 0], mu[:, 1]

    vectorized_gauss = vmap(gauss_elliptic, in_axes=(None, None, 0, 0, 0, None, None))
    norm = vectorized_gauss(pos[:, 0], pos[:, 1], x0, y0, A, sigmax, sigmay)
    norm_model = norm + b[:, None]
    
    return norm_model

@jax.jit
def get_likelihood_gauss_elliptic(params):
    """
    Computes the Chi squared from selected model 
    
    Parameters:
    ----------
    params: list
        selected model parameters
    stamp: arraylike
        image of one star
    
    Returns:
    --------
    summ: float
        chi squared
    """
    model = get_model_gauss_elliptic(params)
    summ = jnp.sum((model - data)**2)
    return summ


@jax.jit
def get_logprior_gauss_elliptic(params):
    """
    Returns the probability to have sigmax and sigma y
    
    Parameters:
    -----------
    params: list
        selected model parameters
    
    Returns:
    -------
    logprior: float
        sum of the two sigma probabilities
    """
    mu, A, b, sigmax, sigmay = params
    logprior_sx = jstats.norm.logpdf(sigmax, loc=1, scale=0.5) # float
    logprior_sy = jstats.norm.logpdf(sigmay, loc=1, scale=0.5) # float
    logprior = logprior_sx + logprior_sy # get the maximize
    return logprior

@jax.jit
def get_logprob_gauss_elliptic(params):
    """ 
    Computes the sum of the sigma probabilities and the chi squared
    
    Parameters:
    -----------
    params: list
        selected model parameters
    stamp: arraylike
        image of one star
    
    Returns:
    logprob: float
        sum of the sigma probabilities and the chi squared
    """
    logprior = -1* get_logprior_gauss_elliptic(params) # to minimize
    likelihood = get_likelihood_gauss_elliptic(params)
    logprob = logprior + likelihood
    return logprob

In [ ]:
nstars = len(percentages)

# guess
x0 = jnp.zeros((nstars,), dtype="float32")
y0 = jnp.zeros((nstars,), dtype="float32")
mu = jnp.vstack([x0,y0]).T
A = jnp.ones((nstars,), dtype="float32")  
b = jnp.zeros((nstars,), dtype="float32")
sigmax = jnp.array(1., dtype="float32")  
sigmay = jnp.array(1., dtype="float32") 

data = percentages.reshape(len(percentages), -1)
grad_func = jax.jit(jax.grad(get_logprob_gauss_elliptic))

In [ ]:
guess_gauss_elliptic = [mu, A, b, sigmax, sigmay]
tncg_params_gauss_elliptic, tncg_loss_gauss_elliptic = fit_tncg(get_logprob_gauss_elliptic, guess_gauss_elliptic, tol=1e-5, niter=100, lmbda=5000)

print("Profilage de la performance")
result_perf = profile_performance(fit_tncg, get_logprob_gauss_elliptic, guess_gauss_elliptic, tol=1e-5, niter=50, lmbda=10000, lines=10)
print("Profilage de la mémoire en Mo")
result_mem = profile_memory(fit_tncg, get_logprob_gauss_elliptic, guess_gauss_elliptic, tol=1e-5, niter=50, lmbda=10000)

## fit Moffat

In [ ]:
def moffat(x, y, x0, y0, A, alpha, gamma):
    r_squared = (x - x0)**2 + (y - y0)**2
    return A * (1 + (r_squared / gamma**2))**(-alpha)


@jax.jit
def get_model_moffat(params):
    """
    Returns the Moffat function

    Parameters:
    -----------
    params: pytree
        [x0,y0: (2,N) centroids
         a: (N,) amplitudes
         b: (N,) backgrounds
         gamma: (1,) float
         alpha: (1,) float]
    
    Returns:
    -------
    model : arraylike
        the Moffat function
    """
    mu, A, b, alpha, gamma = params
    x0, y0 = mu[:, 0], mu[:, 1]
    
    vectorized_moffat = vmap(moffat, in_axes=(None, None, 0, 0, 0, None, None)) # to vectorize on centroids
    norm = vectorized_moffat(pos[:, 0], pos[:, 1], x0, y0, A, alpha, gamma)
    norm_model = norm + b[:, None]
    
    return norm_model
    
@jax.jit
def get_likelihood_moffat(params):
    """
    Computes the Chi squared from the selected model 

    Parameters:
    ----------
    params: list
        selected model parameters
    data: arraylike
        images of stars (flattened)
    pos: arraylike
        The positions (meshgrid) where the model is evaluated.

    Returns:
    --------
    summ: float
        chi squared sum for all stars
    """
    model = get_model_moffat(params)
    summ = jnp.sum((model - data)**2)
    return summ


@jax.jit
def get_logprior_moffat(params):
    """
    Returns the probability to have gamma and alpha
    
    Parameters:
    -----------
    params: list
        selected model parameters

    X: arraylike
        The X coordinates.

    Y: arraylike
        The Y coordinates.
    
    Returns:
    -------
    logprior: float
        sum of the two sigma probabilities
    """
    mu, A, b, alpha, gamma = params
    logprior_alpha = jstats.norm.logpdf(alpha, loc=1.0, scale=0.5) 
    logprior_gamma = jstats.norm.logpdf(gamma, loc=1.0, scale=0.5) 
    logprior = logprior_alpha + logprior_gamma
    return logprior

@jax.jit
def get_logprob_moffat(params):
    """ 
    Computes the sum of the gamma and alpha probabilities and the chi squared

    Parameters:
    -----------
    params: list
        selected model parameters
    data: arraylike
        images of stars (flattened)
    pos: arraylike
        The positions (meshgrid) where the model is evaluated.

    Returns:
    --------
    logprob: float
        sum of the sigma probabilities and the chi squared
    """
    logprior = -1 * get_logprior_moffat(params)  # to minimize
    likelihood = get_likelihood_moffat(params)
    logprob = logprior + likelihood
    return logprob

In [ ]:
# guess
x0 = jnp.zeros((nstars,), dtype="float32")
y0 = jnp.zeros((nstars,), dtype="float32")
mu = jnp.vstack([x0,y0]).T
A = jnp.ones((nstars,), dtype="float32")  
b = jnp.zeros((nstars,), dtype="float32")
alpha = jnp.array(1., dtype="float32")  
gamma = jnp.array(1., dtype="float32") 

data = percentages.reshape(len(percentages), -1)
grad_func = jax.jit(jax.grad(get_logprob_moffat))

In [ ]:
guess_moffat = [mu, A, b, alpha, gamma]
tncg_params_moffat_pg, tncg_loss_moffat_pg = fit_tncg(get_logprob_moffat, guess_moffat, tol=1e-5, niter=100, lmbda=5000)

print("Profilage de la performance")
result_perf = profile_performance(fit_tncg, get_logprob_moffat, guess_moffat, tol=1e-5, niter=50, lmbda=10000, lines=10)
print("Profilage de la mémoire en Mo")
result_mem = profile_memory(fit_tncg, get_logprob_moffat, guess_moffat, tol=1e-5, niter=50, lmbda=10000)